## CHAPTER 10
# Data Aggregation and Group Operations

In [117]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## 10.1 How to Think About Group Operations

In [118]:
df = pd.DataFrame(
    {
        "key1": ["a", "a", None, "b", "b", "a", None],
        "key2": pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
        "data1": np.random.standard_normal(7),
        "data2": np.random.standard_normal(7),
    }
)
df

,key1,key2,data1,data2
0,a,1,-0.663774,-0.374679
1,a,2,0.635923,0.146618
2,None,1,0.087888,0.071936
3,b,2,-2.573686,-0.257555
4,b,1,-1.080465,1.858383
5,a,<NA>,0.158991,0.212065
6,None,1,-0.015779,0.460788


In [119]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [120]:
grouped.mean()

key1
a    0.043713
b   -1.827075
Name: data1, dtype: float64

In [121]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1      -0.663774
      2       0.635923
b     1      -1.080465
      2      -2.573686
Name: data1, dtype: float64

In [122]:
means.unstack()

key2,1,2
key1,,
a,-0.663774,0.635923
b,-1.080465,-2.573686


In [123]:
state = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = np.array([2005, 2005, 2006, 2005, 2006, 2005, 2006])
df["data1"].groupby([state, years]).mean()

CA  2005    0.397457
    2006    0.087888
OH  2005   -1.618730
    2006   -0.548122
Name: data1, dtype: float64

In [124]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,0.043713,-0.005332
b,1.5,-1.827075,0.800414


In [125]:
df.groupby("key2").mean(numeric_only=True)

,data1,data2
key2,,
1,-0.418033,0.504107
2,-0.968882,-0.055469


In [126]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1    -0.663774 -0.374679
     2     0.635923  0.146618
b    1    -1.080465  1.858383
     2    -2.573686 -0.257555

In [127]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [128]:
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [129]:
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [130]:
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


### Iterating over Groups

In [131]:
for name , group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1 -0.663774 -0.374679
1    a     2  0.635923  0.146618
5    a  <NA>  0.158991  0.212065
b
  key1  key2     data1     data2
3    b     2 -2.573686 -0.257555
4    b     1 -1.080465  1.858383


In [132]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', np.int64(1))
  key1  key2     data1     data2
0    a     1 -0.663774 -0.374679
('a', np.int64(2))
  key1  key2     data1     data2
1    a     2  0.635923  0.146618
('b', np.int64(1))
  key1  key2     data1     data2
4    b     1 -1.080465  1.858383
('b', np.int64(2))
  key1  key2     data1     data2
3    b     2 -2.573686 -0.257555


In [133]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"]

,key1,key2,data1,data2
3,b,2,-2.573686,-0.257555
4,b,1,-1.080465,1.858383


In [134]:
pieces

{'a':   key1  key2     data1     data2
 0    a     1 -0.663774 -0.374679
 1    a     2  0.635923  0.146618
 5    a  <NA>  0.158991  0.212065,
 'b':   key1  key2     data1     data2
 3    b     2 -2.573686 -0.257555
 4    b     1 -1.080465  1.858383}

In [135]:
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"}, axis="columns")
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0 -0.663774 -0.374679
1  0.635923  0.146618
2  0.087888  0.071936
3 -2.573686 -0.257555
4 -1.080465  1.858383
5  0.158991  0.212065
6 -0.015779  0.460788
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


C:\Users\sahor\AppData\Local\Temp\ipykernel_7736\695880334.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df.groupby({"key1": "key", "key2": "key",


### Selecting a Column or Subset of Columns

In [136]:
df

,key1,key2,data1,data2
0,a,1,-0.663774,-0.374679
1,a,2,0.635923,0.146618
2,None,1,0.087888,0.071936
3,b,2,-2.573686,-0.257555
4,b,1,-1.080465,1.858383
5,a,<NA>,0.158991,0.212065
6,None,1,-0.015779,0.460788


In [137]:
df.groupby("key1")["data1"]
df.groupby("key1")[["data2"]]

In [138]:
df["data1"].groupby(df["key1"])
df[["data2"]].groupby(df["key1"])

In [139]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1    -0.374679
     2     0.146618
b    1     1.858383
     2    -0.257555

In [140]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped

In [141]:
s_grouped.mean()

key1  key2
a     1      -0.374679
      2       0.146618
b     1       1.858383
      2      -0.257555
Name: data2, dtype: float64

### Grouping with Dictionaries and Series

In [142]:
people = pd.DataFrame(
    np.random.randn(5, 5),
    columns=["a", "b", "c", "d", "e"],
    index=["Joe", "Steve", "Wanda", "Jill", "Trey"],
)
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,0.151040,-1.031924,0.289914,-0.073858,-0.616096
Steve,0.287254,1.036026,1.712924,1.725658,-0.288211
Wanda,-0.279978,NaN,NaN,-1.197041,-1.754968
Jill,-0.655698,0.310660,1.266029,-0.661298,2.474564
Trey,-0.412219,0.552935,0.883716,-0.558404,-0.563093


In [143]:
mapping = {"a": "red", "b": "red", "c": "blue",
            "d": "blue", "e": "red", "f" : "orange"}


In [144]:
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

C:\Users\sahor\AppData\Local\Temp\ipykernel_7736\2628240624.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  by_column = people.groupby(mapping, axis="columns")


,blue,red
Joe,0.216056,-1.496981
Steve,3.438582,1.035070
Wanda,-1.197041,-2.034946
Jill,0.604731,2.129526
Trey,0.325312,-0.422377


In [145]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [146]:
people.groupby(map_series, axis="columns").count()

C:\Users\sahor\AppData\Local\Temp\ipykernel_7736\2727320543.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  people.groupby(map_series, axis="columns").count()


,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### Grouping with Functions

In [147]:
people

,a,b,c,d,e
Joe,0.151040,-1.031924,0.289914,-0.073858,-0.616096
Steve,0.287254,1.036026,1.712924,1.725658,-0.288211
Wanda,-0.279978,NaN,NaN,-1.197041,-1.754968
Jill,-0.655698,0.310660,1.266029,-0.661298,2.474564
Trey,-0.412219,0.552935,0.883716,-0.558404,-0.563093


In [148]:
people.groupby(len).sum()  # group by length of the index value

,a,b,c,d,e
3,0.151040,-1.031924,0.289914,-0.073858,-0.616096
4,-1.067917,0.863595,2.149746,-1.219703,1.911472
5,0.007276,1.036026,1.712924,0.528618,-2.043179


In [149]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,0.151040,-1.031924,0.289914,-0.073858,-0.616096
4,two,-0.655698,0.310660,0.883716,-0.661298,-0.563093
5,one,-0.279978,1.036026,1.712924,-1.197041,-1.754968


### Grouping by Index Levels

In [150]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                     [1, 3, 5, 1, 3]],
                                    names=["cty", "tenor"])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.509882 -0.745809  0.959029  1.152954 -1.119156
1      1.683868  0.569899  0.565350 -0.451333 -0.288012
2     -0.463216  2.116997 -1.197237  1.502190  0.483092
3      0.842362  0.267481  0.621649 -0.125993 -1.416503

In [151]:
hier_df.groupby(level="cty", axis="columns").count()

C:\Users\sahor\AppData\Local\Temp\ipykernel_7736\2795167867.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  hier_df.groupby(level="cty", axis="columns").count()


cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation

In [152]:
df

,key1,key2,data1,data2
0,a,1,-0.663774,-0.374679
1,a,2,0.635923,0.146618
2,None,1,0.087888,0.071936
3,b,2,-2.573686,-0.257555
4,b,1,-1.080465,1.858383
5,a,<NA>,0.158991,0.212065
6,None,1,-0.015779,0.460788


In [153]:
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)

key1   
a     0   -0.663774
      5    0.158991
b     3   -2.573686
      4   -1.080465
Name: data1, dtype: float64

In [154]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.299697,0.586743
b,1,1.493221,2.115938


In [155]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.043713  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0 -1.827075  ...   

                         data2                                         \
           75%       max count      mean       std       min      25%   
key1                                                                    
a     0.397457  0.635923   3.0 -0.005332  0.321533 -0.374679 -0.11403   
b    -1.453770 -1.080465   2.0  0.800414  1.496194 -0.257555  0.27143   

                                    
           50%       75%       max  
key1                                
a     0.146618  0.179341  0.212065  
b     0.800414  1.329399  1.858383  

[2 rows x 24 columns]

### Column-Wise and Multiple Function Application

In [156]:
# tips = sns.load_dataset("tips")
tips = pd.read_csv("examples/tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [157]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [158]:
grouped = tips.groupby(["day", "smoker"])
grouped_pct = grouped["tip_pct"]
grouped_pct.mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [159]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [160]:
grouped_pct.agg([("average", "mean"), ("stdev", "std")])

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [161]:
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [162]:
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [163]:
ftuples = [("Average", "mean"), ("Variance", np.var)]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

C:\Users\sahor\AppData\Local\Temp\ipykernel_7736\4198102554.py:2: FutureWarning: The provided callable <function var at 0x0000021896E07B00> is currently using SeriesGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  grouped[["tip_pct", "total_bill"]].agg(ftuples)


tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [164]:
grouped.agg({"tip": np.max, "size": "sum"})

C:\Users\sahor\AppData\Local\Temp\ipykernel_7736\3453032214.py:1: FutureWarning: The provided callable <function max at 0x0000021896E06E80> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped.agg({"tip": np.max, "size": "sum"})


tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [165]:
grouped.agg({"tip_pct": ["min", "max", "mean", "std"],
            "size": "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [166]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [167]:
tips.groupby(["day", "smoker"], as_index=False).mean(numeric_only=True) # Added numeric_only=True to avoid FutureWarning # শুধু numeric কলাম নাও

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 10.3 Apply: General split-apply-combine

In [168]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [169]:
tips.groupby("smoker").apply(top)

C:\Users\sahor\AppData\Local\Temp\ipykernel_7736\2530541573.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby("smoker").apply(top)


total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [172]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

C:\Users\sahor\AppData\Local\Temp\ipykernel_7736\643835440.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")


total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [173]:
result = tips.groupby("smoker")["tip_pct"].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [174]:
result.unstack()

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [175]:
def f(group):
    return group.describe()
grouped.apply(f)

C:\Users\sahor\AppData\Local\Temp\ipykernel_7736\3509319160.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped.apply(f)


total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]